In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader,Subset
from tqdm import tqdm
import pandas as pd
from torch.optim.lr_scheduler import StepLR

In [20]:
# 데이터 전처리 (CIFAR10은 3채널 이미지)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

# CIFAR10 학습 및 테스트 데이터셋 불러오기
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

# CIFAR10의 targets는 리스트로 되어 있으므로 tensor로 변환
targets = torch.tensor(train_dataset.targets)

num_samples_per_class = 200
selected_indices = []

# 0부터 9까지 각 클래스별로 인덱스를 추출하고, 랜덤하게 num_samples_per_class개 선택
for class_label in range(10):
    # 해당 클래스의 인덱스 추출
    indices = (targets == class_label).nonzero(as_tuple=True)[0]
    # 인덱스를 랜덤하게 섞은 후 원하는 개수만 선택
    selected = indices[torch.randperm(len(indices))[:num_samples_per_class]]
    selected_indices.extend(selected.tolist())

# 선택한 인덱스만 남긴 서브셋 생성
train_dataset = Subset(train_dataset, selected_indices)

In [21]:
class MyNet(nn.Module):
    def __init__(self, GN = False):
        super(MyNet, self).__init__()
        self.GN = GN

        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, padding = 1)
        self.norm1 = nn.BatchNorm2d(16) if not GN else nn.GroupNorm(num_groups = 4, num_channels = 16)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1)
        self.norm2 = nn.BatchNorm2d(32) if not GN else nn.GroupNorm(num_groups = 8, num_channels = 32)
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1)
        self.norm3 = nn.BatchNorm2d(64) if not GN else nn.GroupNorm(num_groups = 8, num_channels = 64)

        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.classifier = nn.Linear(in_features=64 * 4 * 4, out_features=10)

    def forward(self, x):
        x = self.relu(self.norm1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.norm2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.norm3(self.conv3(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [25]:
def experiment(seed):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    batch_sizes = [2, 4, 8, 16, 32, 64, 128]
    Accuracies = {"BatchNorm" : list(), "GroupNorm" : list()}
    num_epochs = 10

    def worker_init_fn(worker_id):
        # 메인 프로세스의 torch.random.seed()로부터 worker에 고유 seed 부여
        worker_seed = torch.initial_seed() % 2**32  # torch.initial_seed()는 메인 seed와 worker_id에 기반하여 결정됨
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    # 매 실험이 동일한 조건에서 시작하기 위해 시드 고정
    print(f"[Seed : {seed}] Experiment Start")

    for batch_size in batch_sizes:
        # 시드 고정
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

        # DataLoader 생성
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True, worker_init_fn = worker_init_fn)
        test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False, num_workers=8, pin_memory=True, worker_init_fn = worker_init_fn)
        models = (model1:=MyNet(GN = False), model2:=MyNet(GN = True))
        scaler = torch.amp.grad_scaler.GradScaler()

        print(f"ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡbatch_size: {batch_size}ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ")
        for model in models:
            model.to(device)
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(model.parameters(), lr=0.0195, momentum=0.91, weight_decay=0.0)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.003)
            #Train Code
            for epoch in range(num_epochs):
                model.train()
                epoch_loss = 0.0
                progress_bar = tqdm(train_loader, desc=f"{'BatchNorm' if not model.GN else 'GroupNorm'} with epoch {epoch + 1}")
                for (data, target) in progress_bar:
                    data, target = data.to(device), target.to(device)
                    optimizer.zero_grad()

                    with torch.amp.autocast(device_type="cuda", dtype=torch.bfloat16):
                        output = model(data)
                        loss = criterion(output, target)
                    epoch_loss += loss.item()
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                    progress_bar.set_postfix({"Loss": loss.item()})
                scheduler.step()
                print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(train_loader)}, lr = {optimizer.param_groups[0]['lr']}")

        for model in models:
            #Test Code
            model.to(device)
            model.eval()
            correct = 0
            with torch.no_grad():
                for _, (data, target) in enumerate(test_loader):
                    data, target = data.to(device), target.to(device)
                    output = model(data)
                    correct += (output.argmax(dim=1) == target).sum().item()
            top1_acc = 100 * correct / len(test_loader.dataset)
            if model.GN == False:
                Accuracies["BatchNorm"].append(top1_acc)
            else:
                Accuracies["GroupNorm"].append(top1_acc)
            print(f"Norm : {'GroupNorm' if model.GN else 'BatchNorm'}, Batch Size: {batch_size}, Top1 Accuracy: {top1_acc:.2f}%")
            del model
            torch.cuda.empty_cache()
        print(f"ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡbatch_size: {batch_size}ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ\n")

    return Accuracies


if __name__ == "__main__":
    results = []
    print("\n\n")
    #Please...
    seed = 84035
    results.append(experiment(seed))




[Seed : 84035] Experiment Start
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡbatch_size: 2ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ


BatchNorm with epoch 1: 100%|██████████| 1000/1000 [00:05<00:00, 197.82it/s, Loss=2.47]


Epoch 1, Loss: 2.5063441915512086, lr = 0.019096216259435018


BatchNorm with epoch 2: 100%|██████████| 1000/1000 [00:05<00:00, 197.05it/s, Loss=2.74]


Epoch 2, Loss: 2.32829296875, lr = 0.017924390203593318


...

GroupNorm with epoch 9: 100%|██████████| 16/16 [00:00<00:00, 19.91it/s, Loss=0.939]


Epoch 9, Loss: 0.8927670046687126, lr = 0.0034037837405649835


GroupNorm with epoch 10: 100%|██████████| 16/16 [00:00<00:00, 19.98it/s, Loss=0.851]

Epoch 10, Loss: 0.8196087554097176, lr = 0.003


Norm : BatchNorm, Batch Size: 128, Top1 Accuracy: 52.05%
Norm : GroupNorm, Batch Size: 128, Top1 Accuracy: 50.09%
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡbatch_size: 128ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ



In [27]:
dfs = []
print(f"[Experiment Result]")
Accuracies = results[0]
differences = [relu - sig for sig, relu in zip(Accuracies["BatchNorm"], Accuracies["GroupNorm"])]

index = [f"batch_size : {i}" for i in [2, 4, 8, 16, 32, 64, 128]]
data = {
    "BatchNorm": Accuracies["BatchNorm"],
    "GroupNorm": Accuracies["GroupNorm"],
    "Differences": np.abs(differences)
}

df = pd.DataFrame(data, index=index)
dfs.append(df)

print(df)

[Experiment Result]
                  BatchNorm  GroupNorm  Differences
batch_size : 2        10.01      30.89        20.88
batch_size : 4        33.81      40.46         6.65
batch_size : 8        44.55      44.27         0.28
batch_size : 16       49.68      48.85         0.83
batch_size : 32       52.16      50.65         1.51
batch_size : 64       53.18      51.37         1.81
batch_size : 128      52.05      50.09         1.96
